###### Coursera  | Andrew Ng's Deep Learning Class | Course2. Improving Deep Neural Networks_Hyperparameter tuning, Regularization and Optimization | Week2. ,,

# [1]. Optimization algorithms

### [1-1]. Mini-batch gradient descent

이번주에는 최적화 알고리즘(optimization algorithm)들에 대해 알아볼 것이다. 전에도 언급했듯 머신러닝을 적용하는 것은 대단히 경험적이고 반복적인 과정이다. 즉 잘 동작하는 모형을 만들기 위해서는 다양한 조건들을 바꿔보고 그때마다 모형을 훈련 시키는 과정이 필요하다. 따라서 모형을 빠르게 훈련시키는 것이 중요하며 빅데이터를 다뤄야 한다면 그 중요성은 두말할 나위 없다. 

데이터 처리를 벡터화 하면 for-loop와 같은 명시적 반복문을 없앨 수 있고, 연산 속도가 빨라진다. 여기서 빨라진다는 것은 $\frac{dw}{dL}(=\text{dw})$를 계산하고 $w$를 한번 수정(one little step of gradient descent)하는 속도가 빨라진다는 것이다. 그런데 데이터 건수가 매우 많아지게 되면 벡터화를 하더라도 $w$를 한번 수정하기까지 시간이 많이 걸릴수밖에 없다. 예를들어 데이터 건수가 5백만건이라면 $w$를 한번 수정하기 위해 5백만건 데이터에 대한 $\text{dw}$를 계산해야 한다. 또한 좋은 $w$값에 수렴할 때 까지 이런 처리를 반복해야 하므로 최적화가 끝날때까지 많은 시간이 소요된다. 

이런 문제에 대한 해결책으로 $w$를 한번 수정하기 위해 전체 데이터에 대한 $\text{dw}$를 계산하는 것이 아니라 일부 데이터에 대한 $\text{dw}$를 계산한 후 $w$를 수정하고, 다음 일부 데이터에 대한 $\text{dw}$를 계산한 후 $w$를 수정하는 것을 반복하는 접근을 생각해볼 수 있다. 이때 일부 데이터(baby training set)를 mini-batch라 한다. 예를들어 전체 5백만건 데이터가 있다고 할 때 mini-batch 크기가 1000이면 전체 데이터는 총 5000개의 mini-batch로 나뉘게 된다. 이 경우 1번째부터 1000번째 샘플에 대한 $\text{dw}$를 계산하여 $w$를 수정하고, 1001번째 샘플부터 2000번째 샘플에 대한 $\text{dw}$를 계산하고 $w$를 수정하는 과정을 5000번 하게 된다. 


전체 데이터 세트에서 $X$에 대한 $1, 2, i, \cdots$번째 mini-batch를 표현하기 위해 $X^{\{1\}}, X^{\{2\}}, \cdots, X^{\{5000\}}$와 같이 중괄호(curly brace)를 사용한 표기법을 사용할 것이고, 마찬가지로 $Y$에 대한 mini-batch를 $Y^{\{1\}}, Y^{\{2\}}, \cdots, Y^{\{5000\}}$와 같이 표기할 것이다. 그래서 $t$번째 mini-batch는 $X^{\{t\}}$와 $Y^{\{t\}}$ 의 쌍으로 구성된다. 


앞서 강의에서부터 등장했던 표기법들을 정리해보면       
$x^{i}$는 i번째 샘플을,      
$z^{[l]}$은 l번째 레이어의 z값을     
$X^{\{t\}}$는 t번째 mini-batch의 X를 의미한다.

또한 $t$번째 mini-batch, $X^{\{t\}}$의 shape은 ($n_x$, 1000)이고 $Y^{\{t\}}$의 shape은 (1, 1000) 이다.

이렇게 데이터를 나누고 gradient descent로 최적화 하는 것을 **mini-batch gradient descent**라 한다.(반면 mini-batch가 아니라 전체 데이터를 한번에 처리하는 것을 **batch gradient descent**라 한다.) 


![Local image](./images/C2W2L01_02.PNG)

mini-batch gradient descent를 어떻게 구현하는지 알아보자. 

전체 데이터가 5백만건이고 mini-batch 크기가 1000이라면 총 5000개의 mini-batch가 있다. 우선 첫번째 mini-batch(1번째부터 1000번째 샘플)를 이용하여 (각 레이어를 따라 출력레이어까지) 순전파($A^{[1]}, \cdots, A^{[L]}$)를 계산한다. 다음으로 cost값 $J$를 구한다. 이제 출력레이어에서부터 입력레이어까지 역전파를 계산하여 결국 $\text{dw}$를 구하고 $w$를 업데이트 한다. 이때 순전파와 역전파 계산은 벡터화할 수 있고, 이런 과정을 5000개의 mini-batch에 대하여 진행한다. 즉 5백만건 데이터를 한번 처리하게 되면 $w$를 5000번 수정하는 것이 된다. 이렇게 전체 데이터를 한번 처리하는 것을 epoch 1번(single path through training set)이라 하며, 최적의 w에 수렴하기까지 여러번의 epoch(미국식 발음은 에펙, 영국식 발음은 이파크)를 반복하게 된다. 


![Local image](./images/C2W2L01_01.PNG)

### [1-2]. Understanding mini-batch gradient descent

일반적으로 batch gradient descent를 이용하면 매 epoch마다 cost가 감소한다. 만약 cost가 증가한다면 learning rate이 너무 크다거나 어떤 다른 문제를 의심해 봐야 한다. 

반면 mini-batch gradient descent의 경우 매 mini-batch를 처리할 때마다 cost가 증가할수도 있고 감소할 수도 있다 다만 전체적으로는 감소하는 경향을 갖는다. 이런 특징은 전체 데이터에 대한 여러 mini-batch 마다 전체 데이터의 특성을 잘 반영하는 데이터가 뽑힌 경우도 있지만 그렇지 않은 경우도 있기 때문에 발생한다. 

![Local image](./images/C2W2L02_01.PNG)

mini-batch방식을 사용하려면 mini-batch의 크기를 어느정도로 할지를 결정해야 한다. 
만약 mini-batch 크기가 $m$이라면 이건 batch gradient descent와 같고,
mini-batch 크기가 $1$이라면 이건 stochastic gradient descent가 된다. mini-batch의 크기에 따른 $w$좌표의 궤적을 살펴보면 이들 각각의 특징을 알 수 있다. 

<u>(아래 슬라이드의 보라색 궤적)</u> 우선 batch gradient descent의 경우 각 step의 크기가 크고 global minimum으로 거의 직행하는 특징이 있다. 이는 각 step의 방향에 노이즈가 매우 작다고 할 수 있다. 또한 각 step을 계산하기 위해 전체 데이터를 처리해야하므로 각 step이 상대적으로 오래 걸린다.

<u>(아래 슬라이드의 파란색 궤적)</u> 반면 stochastic gradient descent의 경우 각 step의 크기가 매우 작고 그 뱡항이 매우 자주 바뀌고 꼭 minimum으로 향하지 않을 수도 있다. 이는 각 step의 방향에 노이즈가 매우크다고 할 수 있다. 또한 각 step을 계산하는 것은 오래 걸리지 않으나 전체 데이터를 벡터화 하지 않고 한건 한건 계산하는 것이므로 계산 효율이 좋지 않다. 그래서 전체적으로는 처리 속도가 매우 느리다.

<u>(아래 슬라이드의 녹색 궤적)</u> 이렇게 mini-batch 크기를 1이나 m과 같이 극단적인 값이 아니라 그 중간의 적절한 값을 사용한다면 각 mini-batch에 대한 계산을 벡터화 할 수 있고, 전체 데이터를 다 처리하지 않고도 썩 괜찮은 방향으로 gradient step을 밟아 갈 수 있다. 

![Local image](./images/C2W2L02_02.PNG)

그렇다면 적절한 mini-batch size는 어느정도 일까? 대략적인 가이드라인으로는 만약 데이터 크기가 작다면(대략 2000건 이하?) 그냥 batch gradient descent를 사용하는 것이 좋다. 반면 데이터 크기가 이보다 크다면 mini-batch gradient descent를 사용하는 것이 좋고 mini-batch size는 64, 128, 256, 512 등 2의 차수를 사용하는 것을 추천한다. 
그리고 mini-batch size를 정할 때 한번에 처리하는 mini-batch data $X^{\{t\}}$와 $Y^{\{t\}}$가 CPU/GPU 메모리에 들어올 정고가 되어야 한다. 이보다 크다면 메모리 부족 으로 인해 처리 속도가 급격히 저하되는 문제를 겪게 될 것이다. 

이렇게 mini-batch size또한 하나의 hyper parameter라 할 수 있고, 다양한 값을 실험해보고 적절한 값을 선택해야 하는 것이다.

![Local image](./images/C2W2L02_03.PNG)

### [1-3]. Exponentially weighted averages, 지수가중평균

gradient descent보다 나은 최적화 알고리즘이 여럿 있는데, 이들을 이해하기 위해 우선 '지수가중 평균'(exponentially weighted average)에 대해 알 필요가 있다. (통계학에서는 지수 가중 이동 평균exponentially weighted 'moving' average라 부른다.)

아래와 같이 반년간 런던의 매일 특정 시간대의 기온이 있다고 하자. 그래프에서 알 수 있는 것 처럼 봄에는 온도가 낮았다가 여름으로 가면서 온도가 올라가고 여름이 지나면서 온도가 다시 내려가는 경향이 있다. 

1일자 기온부터 시작하여 지수가중평균을 구해보자. 구하는 방식은 t번째 날짜의 지수가중평균을 구한다면 t-1번째 날짜까지의 지수가중평균 $v_{t-1}$에 가중치 $\beta$를 곱하고, t번째 날짜의 기온 $\theta_t$에 $1-\beta$의 가중치를 곱하고 두 값을 더한다. 

![Local image](./images/C2W2L03_01.PNG)

$t$ 시점의 기수 가중 평균은 아래와 같으며
$v_t = \beta v_{t-1} + (1-\beta) \theta_t$

이렇게 구한 지수가중평균 $v_t$는 근사적으로 $\frac{1}{1-\beta}$일 동안의 값들에 대한  평균을 구한 것과 같다. 예를들어 $\beta=0.9$라면 대략 10일간의 기온의 평균을 구한 것과 같고, 만약 $\beta=0.98$이라면 대략 50일간의 기온의 평균을 구하는 것과 같다. 
이들 지수가중평균을 그려보면 $\beta$값이 1에 가까워질수록 실제 값보다 오른쪽으로 이동되어 있는 것을 알 수 있다. 이는 $\beta$값이 커질수록 과거 데이터에 대한 가중치가 커지므로 최근 데이터가 반영되기까지 시간이 걸리는 것이다. 반면 $\beta$값이 작아지면 매일의 데이터에 가까워지지만 최근 데이터를 많이 반영하므로 최근 데이터의 노이즈가 많이 반영된다. 

![Local image](./images/C2W2L03_02.PNG)

### [1-4]. Understanding exponentially weighted averages

지수 가중 평균에 대해 좀더 알아보도록 하자. 
$t$시점의 지수가중평균은 
$v_t = \beta v_{t-1} + (1-\beta) \theta_t$
인데, 런던의 기온 데이터는 아래 슬라이드의 파란 점과 같은데, $\beta$가 0.98이면 지수 가중 평균은 노란 선과 같고, $\beta$가 0.9이면 빨간 선과 같고, $\beta$가 0.5이면 녹색 선과 같다. 즉 $\beta$가 작아질수록 실제 데이터와 가깝고 노이즈를 그대로 반영하게 되고, $\beta$가 커질수록 노이즈가 줄어들면서 실제 데이터와의 딜레이가 발생하게 된다.

![Local image](./images/C2W2L03b_01.PNG)

예를들어 $\beta$가 0.9일 때 100번째 시점의 지수 가중 평균을 구해보자.
$\begin{align} v_{100} &= 0.1 \theta_{100} + 0.9 v_{99} \\
&= 0.1 \theta_{100} + 0.1 \cdot 0.9 \cdot \theta_{99} + 0.1 \cdot 0.9^2 \cdot \theta_{98} + 0.1 \cdot 0.9^3 \theta_{97} + \cdots
\end{align}$

위 식을 보면 $t$시점으로부터 멀리 있는 $\theta_{?}$일수록 더 높은 차수의 $\beta=0.9$를 곱하고 있다. (즉 $0.1$, $0.1 \cdot 0.9$, $0.1 \cdot 0.9^2$, $0.1 \cdot 0.9^3$, ... )

참고로 이 계소값들은 모두 더하면 1이 된다.($\sum_{i=1}^{\inf}0.1 \cdot 0.9^{i-1} = \frac{0.1}{1-0.9}= 1$)


또한 $(1- \epsilon)^{1/ \epsilon} = \frac{1}{\epsilon}$이고 이 값은 대략 0.35이다. $\beta$가 0.9인 경우 이 계산은 $0.9^{19} \approx 0.35$가 되며 10일정도전 데이터에 대한 가중치는 $t$시점 데이터에 비해 1/3 수준이라는 것이다. 이런 이유로 $\beta$가 0.9이면 대략 과거 10일간의 데이터를 평균낸 것이라고 예기한다.
만약 $\beta$가 0.98일 경우 위 계산은 $0.98^{1/0.02}$로서 대략 1/0.02=50일간의 평균이 되는 것이다. (대략 그렇게 생각하면 된다는 것이지 정확히 50일간의 평균은 아니다.)

![Local image](./images/C2W2L03b_02.PNG)

실제 구현에 있어서는 $v_0$, $v_1$, $v_2$, ...를 별도의 변수로 잡지 않고 아래와 같이 한 변수를 이용해 for-loop로 구현할 수 있다.
$$\begin{align}
&\text{Repeat} \{ \\
&\qquad \text{Get next} ~~ \theta_t \\
&\qquad   v := \beta v + (1-\beta) \theta_t \\
\}
\end{align}$$
이렇게 구현하면 모든 시점의 $v_t$를 메모리에 저장할 필요가 없는 장점이 있다..

![Local image](./images/C2W2L03b_03.PNG)

### [1-5]. Bias correction in exponentially weighted average

아래 슬라이드에서 $\beta$가 0.98일때 지수 가중 평균은 보라색 선과 같은데, 초기 값이 실제 값과 큰 차이가 있음을 알 수 있다. 이 차이는 지수 가중 평균의 초기값이 0이라서 발생하는 것인데, 이런 부정확함을 보정해주는 'Bias correction'이라는 방법이 있다. 

예를들어 $\beta$가 0.98일 때, 
$v_0 = 0$이고
$v_1 = 0.98 \cdot v_0 + 0.02 \cdot \theta_1 = 0.02 \cdot \theta_1$이다. 만약 첫 데이터 $\theta_1$가 100이라면 $v_1$은 겨우 2이다. 또한  $v_2 = 0.0196 \cdot \theta_1 + 0.02 \cdot \theta_2$로서 ($\theta_1$과 $\theta_2$가 모두 양수일 때) $\theta_2$보다 매우 작다. 
이런 차이를 보정하기 위해 $v_t$대신 $\frac{v_t}{1-\beta^t}$를 사용하는 방법을 사용할 수 있다. 즉 t가 작을 때에는 $v_t$에 1보다 큰 어떤 값($\frac{1}{1-\beta^t}$)이 곱해지고, t가 증가하면서 보정 값이 1로 수렴하는 효과가 나타난다. 

![Local image](./images/C2W2L04_01.PNG)

### [1-6]. Gradient descent with momentum

모멘텀(momentum) 알고리즘 혹은 gradient descent with momemtum이라 부르는 최적화 알고리즘이 있다. 이는 거의 항상 보통의 gradient descent보다 빠른 최적화 성능을 보인다. 
이 알고리즘을 한마디로 표현하자면 gradient($\frac{dL}{dw}(=\text{dw})$)의 지수 가중 평균(exponentially weighted average)을 구하고, $\text{dw}$대신 이 값을 weight 업데이트에 사용하는 것이다.

예를들어 weight이 단 두개이고 cost-contour가 아래 그림과 같이 중심에 global minimum이 있고 위 아래로 찌그러진 형태 다고 하자. 이때 일반적인 gradient descent로 최적화를 하면 아래 파란색 궤적과 같이 contour의 양 사이드에서는 지그제그 형태로 최적화가 진행된다. 사실 이런 형태는 곧장 global minimum으로 가는것에 비하면 매우 비 효율적이며 learning rate이 클 경우 cost가 증가하는 일도 발생하게 된다.  사실 contour가 이런 형태인 경우 세로축 방향으로는 조금만 움직이고 가로축 방향으로는 많이 움직이는 것이 바람직할 한데, 보통의 gradient descent로는 이걸 표현하는 것이 쉽지 않다. 

momentum방법은 이 문제를 어떻게 접근하는지 살펴보자.

우선 batch혹은 mini-batch에 대해 dw와 db를 구한다.
(보통의 gradient descent에서는 아래와 같이 w와 b를 업데이트 했을 것이다.)
$$w := w - \alpha \text{dw} \\
b := b - \alpha \text{db}$$

그리고 이전 batch(epoch)들 혹은 이전 mini-batch들에서 구했던 값들을 이용해 dw와 db의 지수 가중 평균을 구한다. 
$$
v_{dw} := \beta v_{\text{dw}} + (1-\beta) \text{dw} \\
v_{db} := \beta v_{\text{db}} + (1-\beta) \text{db}
$$
이렇게 구한 지수 가중 평균값을 이용해 w와 b를 업데이트 한다.
$w := w - \alpha v_{\text{dw}}$, 
$b := b - \alpha v_{\text{db}}$

이것이 어떤 효과를 갖을지 생각해보자. 어떤 w방향에 대해 각 gradient step마다 dw의 부호가 바뀐다면, 일반적인 gradient descent에서는 그 궤적이 지그-제그 형태가 될 것이다. 이런 경우 이들 dw들의 평균을 구하면 양수와 음수가 상쇄되어 $v_{dw}$는 0에 가까운 값이 된다. 따라서 지그-제그 흔들림이 줄어드는 효과를 갖게 된다.

모멘텀 방법을 물리학 관점에서 생각해보면 
$\text{dw}$와 $\text{db}$는 가속도(acceleation), $v_{dw}$와 $v_{db}$는 속도(velocity), $\beta$는 (1보다 작은 값이므로) 속도를 제한하는 역할을 하게 된다.

gradient descent에서는 각 gradient step이 독립적이었다면 모멘텀 방법에서는 각 gradient step의 $\text{dw}$와 $\text{db}$가 가중 평균으로 이후 값에 영향을 주게 된다.

![Local image](./images/C2W2L05_01.PNG)

실제 구현은 아래와 같은데, 결정해야 하는 hyper parameter는 learning rate $\alpha$와 평균 가중치 $\beta$이다. 일반적으로 $\beta$값으로 0.9를 많이 사용한다. 또한 가중평균을 구할 때 bias correction을 사용할 수도 있다. 
그리고 종종 $(1-\beta)$항을 생략하는 구현도 있는데, 이 경우 $v_{dw}$가 $(1-\beta)$만큼 커지는 효과가 있기 때문에 결과적으로 learning rate alpha가 커지는 효과를 갖는다. 이때문에 $\alpha$를 이에 맞게 다시 튜닝해야하는 상황이 된다. 이런 이유 때문에 $(1-\beta)$를 생략하지 않는 버전이 좀 더 직관적이다. 

![Local image](./images/C2W2L05_02.PNG)

.

### [1-7]. RMSProp

.

![Local image](./images/C2W2L06_01.PNG)

.

.

### [1-8]. Adam optimization algorithm

.

![Local image](./images/C2W2L07_01.PNG)

.

![Local image](./images/C2W2L07_02.PNG)

.

### [1-9]. Learning rate decay

.

![Local image](./images/C2W2L08_01.PNG)

.

![Local image](./images/C2W2L08_02.PNG)

.

![Local image](./images/C2W2L08_03.PNG)

.

.

### [1-10]. The problem of local optima

.

![Local image](./images/C2W2L09_01.PNG)

.

![Local image](./images/C2W2L09_02.PNG)

.